
# ASDC SSWP Colour Picker Notebook

Preparatory steps and workflow for Colour Picker functionality of the ASDC/SSWP API 

Developed by 2Pi Software

Dane Evans

## Function of the SSWP Colour Picker

The SSWP Colour Picker examines sets of images to identify pixels with colours falling in a specified range. To assist interaction of determining the colour range, a sample colour 'Swatch' is referenced.

The Swatch may be a small cropped area from a target image, a composite of selected target pixel spreads, or a generated colour gradient.

When the Colour Picker is run, positive image results will be returned with bounding boxes marked. Bounding box extents as image co-ordinates are also output as text.

Colour ranges can be assessed in either RGB or HSV colour spaces. For the swatch to determine the range, it is first subject to clustering, establishing a majority colour spread, then the calculated spread is bracketed. (Defaulting to two bands of clustering as foreground vs background and retaining 98% spread of the majority band.)

Colour Picking and range extraction from Swatches can be chained into a single automated run or can be run as distinct processes.

In [ ]:
# --- Initialisation ----

# Imports 
import requests 
import json
import time
from getpass import getpass
import ipywidgets as widgets

# For dev, allow refresh of support lib
import importlib

import ASDC_API_support
importlib.reload(ASDC_API_support)
print("Lib loaded")

# Useful links to remember
site_url = "https://dev2pi.sswp.cosinecrm.com.au"
base_url = site_url + "/sswpapps-api/"
auth_url = base_url + "auth"



# Establish API authorisation

## Setup your ASDC user access

1. go to https://dev2pi.sswp.cosinecrm.com.au
2. log in to your SSWP account 
3. go to SSWP Apps > Manage account
4. set an App password using the Update Password interface in the middle of the screen



## Obtain a token for API access
1. go to https://dev2pi.sswp.cosinecrm.com.au/sswpapps-api/auth
2. log in with your ASDC user name and password
3. copy out the token, and paste it into the following cell as a string
4. (revisit the link to refresh or revoke your token at any time)


# Use a token to access API endpoints



In [ ]:
#@title
# Initialise the API 

token = getpass("Paste your ASDC API token here:")     
asdc_api = ASDC_API_support.ASDC_API_wrappers(token)



## The Info endpoint describes all available actions.

In [ ]:
asdc_api.format_human_readable(asdc_api.info_request())

## Example: The Uploads endpoint summarises your stored files.

In [ ]:
uploads = asdc_api.asdc_uploads()
uploads = uploads.json()['response']['payload']['uploads']
list_uploads = []
for i in uploads:
  list_uploads.append(str(i.get('id')) + ": " + str(i.get('title')))

upload_picker = widgets.Dropdown(options=list_uploads)
upload_picker

# Workflows



## To make image sets available
- Nominate images for upload using 'public facing' URI's
- Upload sets of images including colour range swatches
- Keep track of your uploads by title and id


In [ ]:
## Enter URI's, whitespace separated or on multiple lines
#  Enter a blank last line to finish
uploading_uri_set = []
while True:
  uri_block = input()
  if uri_block == "":
    break
  uri_block = uri_block.strip()
  upload_additions =  uri_block.split()
  [uploading_uri_set.append(y) for y in (upload_additions)]

print("Confirming upload URI's:")
print(uploading_uri_set)


In [ ]:
## Start the upload process 
process_title = input("Enter a unique upload title: ")

process = asdc_api.asdc_processes_source(process_title, uploading_uri_set)

if process.status_code != 500:
  process_ids = []
  for i in process.json()['response']['payload']['launched']:
    process_ids.append(int(i.get('id')))
  print(process_ids)

  # Poll for the processes to finish 
  for i in range(30):
    print("Polling: " + str(i))
    done = True
    for process in process_ids:
      # explicitly note the process type as "source" for status of upload
      refresh = asdc_api.asdc_process_refresh(str(process)+"/source")
      message = refresh.json()['response']['payload']['process'][0]['message']
      print("Process " + str(process) + ": " + message)
      done &= ("Uploaded" in message)
    if done: 
      print("Processes completed. ")
      i=0
      break
    time.sleep(1)
  if (i==29):
    print("Processing is slow, please check SSWP console.")
else: # 500 error
  print(process.reason)
  print(process.json()['response']['payload'])


## To run pixel colour matching against a colour range swatch
- List your uploads
- Select an upload by title and id
- Nominate the included colour range swatch 
- Start a "chain" process 
- Poll until it process completes
- Get back marked image files and bound box data 

In [ ]:
## Get a list of your uploads 

uploads = asdc_api.asdc_uploads()
#asdc_api.format_human_readable(uploads)
uploads = uploads.json()['response']['payload']['uploads']
list_uploads = []
for i in uploads:
  list_uploads.append(str(i.get('id')) + ": " + str(i.get('title')))

print("Please select an upload:")
upload_picker = widgets.Dropdown(options=list_uploads)
upload_picker

In [ ]:
## After selecting an upload above, select an image to be a swatch 
selected_upload = upload_picker.value.split(':')[0]

# get the file list from the selected upload
upload_details = asdc_api.asdc_uploads(selected_upload, get_data=True)
#asdc_api.format_human_readable(upload_details)
files = upload_details.json()['response']['payload']['uploads'][0]['results']
list_files = []
for i in files:
  list_files.append(str(i.get('file')) + ": " + str(i.get('filename')))

print("Please select a swatch image:")
swatch_picker = widgets.Dropdown(options=list_files)
swatch_picker

In [ ]:
## Start the chained processes
process_title = input("Enter a unique processing title: ")

selected_swatch = swatch_picker.value.split(':')[0]

# Adapt parameters if you wish
colorspace = "BGR"
bands = 2
spread = 82

process = asdc_api.asdc_processes_chain(process_title, selected_swatch, selected_upload, colorspace, bands, spread)
#print(process.json())

if process.status_code != 500:
  process_ids = []
  for i in process.json()['response']['payload']['launched']:
    process_ids.append(int(i.get('id')))
  print(process_ids)

  # Poll for the processes to finish 
  for i in range(30):
    print("Polling: " + str(i))
    done = True
    for process in process_ids:
      refresh = asdc_api.asdc_process_refresh(process)
      message = refresh.json()['response']['payload']['process'][0]['message']
      print("Process " + str(process) + ": " + message)
      done &= message == "Results available"
    if done: 
      print("Processes completed. ")
      i=0
      break
    time.sleep(1)
  if (i==29):
    print("Processing is slow, please check SSWP console.")
else: # 500 error
  print(process.reason)
  print(process.json()['response']['payload'])


In [ ]:
## view results data 
for process in process_ids:
  print(process)
  asdc_api.format_human_readable(asdc_api.asdc_runs(process, get_data=True))


In [ ]:
## Chase links from results per above
result_link_details = asdc_api.asdc_runs(process_ids[1], get_data=True)
files = result_link_details.json()['response']['payload']['runs'][0]['results']

list_files = []
for i in files:
  list_files.append(str(i.get('get_link')))
print("Please select a results download request:")
link_picker = widgets.Dropdown(options=list_files)
link_picker


In [ ]:
## Get a directly accessible link:
print(asdc_api.asdc_download_by_link(request_url=link_picker.value).content)